In [1]:
import json
from sympy import sympify
import sympy as sp
import torch
from torchengine import AnalyticalSetSympy, Function, EliminateAnalysis, ParallelAnalysis, EliminateAnalysisMergeResiduals
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})
import re
from scipy.optimize import minimize

def extract_number(string):
    return int(re.search(r'\d+', string).group())

def process_expression(exprstr, symb_str_mapping):
    exprsympy = sympify(exprstr, locals=symb_str_mapping)
    # free_symbols returns a set which is not ordered
    sorted_symbols = sorted(exprsympy.free_symbols, key=lambda s: s.name)
    for symbol in sorted_symbols:
        if str(symbol) not in symb_str_mapping:
            symb_str_mapping[str(symbol)] = symbol
    return exprsympy

def process_json(data, symb_str_mapping=None):
    functional_sets = data["functional_sets"]
    
    symb_str_mapping = symb_str_mapping if symb_str_mapping else {}
    analysismap = []
    
    for functional_set in functional_sets:
        analysis_str = functional_set['analysis']
        output_var_str= functional_set['functionalvar']
        analysis = process_expression(analysis_str, symb_str_mapping)
        if output_var_str not in symb_str_mapping:
            outputvar = sp.Symbol(output_var_str)
            symb_str_mapping[output_var_str] = outputvar
        else:
            outputvar = symb_str_mapping[output_var_str]
        analysismap.append((analysis, outputvar))
    
    return analysismap, symb_str_mapping

In [2]:
# check if file exists or not return True/False
import os
def load_file(file_name):
    file_path = f'../applications/data/{file_name}.json'
    os.path.isfile(file_path)
    with open(file_path, 'r') as file:
        json_str = file.read()
    data = json.loads(json_str)
    return data

In [3]:
symb_str_mapping = {}
all_analyses = {}
disciplines = ['pearl_geom','pearl_hydro', 'pearl_mass','pearl_prop','pearl_comms','pearl_power', 'pearl_solar', 'pearl_battery']
flattened_output = []
equality_constraints_sympy = []
for file_name in disciplines:
    data = load_file(file_name)
    flattened_output += data['functional_sets']
    equality_constraints_sympy += [process_expression(elt, symb_str_mapping) for elt in data.get('equality_constraints',[])]
    analysismap, symb_str_mapping = process_json(data, symb_str_mapping)
    all_analyses[file_name] = analysismap

In [4]:
indices = {elt: torch.tensor([int(i)]) for i, elt in 
            enumerate(symb_str_mapping.values())}
sets ={}
for file_name, analysismap in all_analyses.items():
        sets[file_name] = {idx:AnalyticalSetSympy(analysis, outputvar=outputvar, indices=indices) for idx,(analysis,outputvar) in enumerate(analysismap)}
equality_constraints = [Function((
        sorted(expr.free_symbols, key=lambda s: s.name),
        sp.lambdify(sorted(expr.free_symbols, key=lambda s: s.name), expr, torch),  
        ), indices=indices) for expr in equality_constraints_sympy]

### Partitioned analysis

In [5]:
partition_order = [{1,2,3}, {4,5}, {6}] # a feedforward partition is equivalent to splitting up the sets, however the shared vars will be different depending on the partition (in the extreme all vars will be shared)

In [6]:
custom_partition_order = []
flat_sets = {}
for file_name, subsets in sets.items():
    custom_partition_order.append(())
    for elt in subsets.values():
        flatidx = len(flat_sets)
        flat_sets[flatidx] = elt
        custom_partition_order[-1] += (flatidx,)

In [7]:
partition_order = custom_partition_order
#partition_order = [{idx} for idx in flat_sets.keys()]

In [8]:
sequential = {}
for partidx, elt in enumerate(partition_order):
    if len(elt) ==1:
        sequential[partidx] = flat_sets[next(iter(elt))].analysis
    else:
        sequential[partidx] = EliminateAnalysis([flat_sets[idx].analysis for idx in elt], [])

In [9]:
xvalsdict = load_file('pearl_params')
x0 = torch.zeros(len(indices), dtype=torch.float64)
for key, val in indices.items():
    x0[val] = xvalsdict[str(key)]

In [10]:
x0.numpy()

array([1, 2, 0.2, 0.3, 1.25, 2, 0.45, 0.5, 2, 1, 9.8, 1000, 1, 1, 1, 0.5,
       1, 1, 1, 1, 1, 1, 1, 1, 10, 1, 1, 50, 50, 1, 700, 2700, 1, 1, 0.75,
       1, 1, 300000000, 2200000000, 1, 27, 32, 1, 0.55, 1, 5.3, 1,
       6378000, 600000, 1, 1, 38.9, 0.93, 0.79, 0.93, 85000000, 135, 0, 1,
       3600, 1, 50, 86400, 1, 3600, 1, 1900, 1, 1, 43200, 1, 1, 1, 43200,
       1, 0.9, 1, 0.05, 0.27, 800, 0.96, 0.7, 1, 0.85, 1, 5, 720000])

In [11]:
def custom_formatter(x):
    if abs(x) < 0.01:
        return "{:.2e}".format(x).replace('+0', '')
    elif abs(x) > 10000:
        return "{:.2e}".format(x).replace('+0', '')
    else:
        return "{:.3f}".format(x).rstrip('0').rstrip('.')

In [12]:
matrix =([str(k) for k in indices.keys()], x0, sequential[0](x0))
col_widths = [max(len(custom_formatter(row1)), len(custom_formatter(row2)), len(name)) for name,row1,row2 in zip(*matrix)]
print(" ".join("{:<{}}".format(name, width) for name, width in zip(matrix[0], col_widths)))
for row in matrix[1:]:
    print(" ".join("{:<{}}".format(custom_formatter(num), width) for num, width in zip(row, col_widths)))

A_solar D_f alpha d   D_d  D_s h_f  t_d t_s V_d g   rho_w F_B F_W K_B t_f K_G I B_M G_M C_33 A_33 m_platform \omega_0 eta_solar m_solar m_batt m_comms m_prop m_struct rho rho_h S_w C_d eta_m v P_move c      f      l e_t theta_t L_pt eta_parab G_t D_r G_r R_e    h      S L_s E_N  L_a   L_l  L_p   R      T_s k        P_comms t_move E_move P_hotel t_mission E_hotel t_comms E_comms E_AUV G E_service t_service P_service E_required E_recharge t_recharge P_recharge I_deg L_solar d_deg eta_s phi_s theta DOD N_batt eta_battery C m_battzero mu_battery
1       2   0.2   0.3 1.25 2   0.45 0.5 2   1   9.8 1000  1   1   1   0.5 1   1 1   1   1    1    1          1        10        1       1      50      50     1        700 2700  1   1   0.75  1 1      3.00e8 2.20e9 1 27  32      1    0.55      1   5.3 1   6.38e6 6.00e5 1 1   38.9 0.933 0.79 0.933 8.50e7 135 1.38e-23 1       3600   1      50      8.64e4    1       3600    1       1900  1 1         4.32e4    1         1          1          4.32e4     

### Get shared variables

In [13]:
sharedvars = []
for _, s1 in sequential.items():
    for _, s2 in sequential.items():
        svars = [elt1 for elt1 in s1.structure[1] for elt2 in s2.structure[0] if elt1 == elt2 and elt1 not in sharedvars]
        sharedvars+=svars
print(sorted(torch.tensor(sharedvars).tolist()))

[0, 3, 7, 13, 22, 26, 36, 58, 71]


### Get feedback vars

In [14]:
all_sequential = EliminateAnalysis(list(sequential.values()), [])
couplingvars = [elt for elt in all_sequential.structure[0] if elt in all_sequential.structure[1]]
print(sorted(torch.tensor(couplingvars).tolist()))

[0, 7, 22, 26]


In [15]:
for key,val in sequential[5].structure_full.items():
    print(key,val)

60 [59, 36]
63 [61, 62]
65 [64, 58]
68 [66, 67]
70 [66, 67, 69]
71 [61, 62, 66, 67, 59, 36, 64, 58]


In [16]:
couplingvars = [0,7,26]

In [17]:
for key,val in all_sequential.structure_full.items():
    print(key, val)

3 [2, 1]
9 [8, 6, 4, 5, 7, 1]
12 [10, 11, 8, 6, 4, 5, 7, 1]
13 [10, 11, 8, 6, 4, 5, 7, 1]
14 [8, 4, 1, 5, 6, 7]
16 [8, 4, 1, 15, 5, 7]
17 [1]
18 [1, 8, 6, 4, 5, 7]
19 [1, 8, 6, 4, 5, 7, 15]
20 [10, 1, 11]
21 [5, 4, 1, 11]
23 [10, 1, 11, 5, 4, 22]
22 [10, 11, 8, 6, 4, 5, 7, 1]
25 [0, 24]
29 [28, 0, 24, 26, 10, 11, 8, 6, 4, 5, 7, 1, 27]
7 [8, 4, 30, 1, 15, 5, 28, 0, 24, 26, 10, 11, 6, 7, 27, 31]
32 [5, 6, 4, 8, 30, 1, 15, 28, 0, 24, 26, 10, 11, 7, 27, 31]
36 [5, 6, 4, 8, 30, 1, 15, 28, 0, 24, 26, 10, 11, 7, 27, 31, 34, 33, 35]
39 [37, 38]
42 [40, 41]
44 [37, 38, 2, 1, 43, 40, 41]
46 [37, 38, 45, 43]
49 [47, 48]
50 [37, 38, 47, 48]
58 [37, 38, 2, 1, 43, 40, 41, 45, 57, 55, 54, 53, 51, 56, 52, 47, 48]
60 [59, 5, 6, 4, 8, 30, 1, 15, 28, 0, 24, 26, 10, 11, 7, 27, 31, 34, 33, 35]
63 [61, 62]
65 [64, 37, 38, 2, 1, 43, 40, 41, 45, 57, 55, 54, 53, 51, 56, 52, 47, 48]
68 [66, 67]
70 [66, 67, 69]
71 [61, 62, 66, 67, 59, 5, 6, 4, 8, 30, 1, 15, 28, 0, 24, 26, 10, 11, 7, 27, 31, 34, 33, 35, 64, 37, 3

In [18]:
list(indices.keys())[23]

\omega_0

In [19]:
all_sequential.structure[0] 

tensor([ 0,  1,  2,  4,  5,  6,  7,  8, 10, 11, 15, 22, 24, 26, 27, 28, 30, 31,
        33, 34, 35, 37, 38, 40, 41, 43, 45, 47, 48, 51, 52, 53, 54, 55, 56, 57,
        59, 61, 62, 64, 66, 67, 69, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85,
        86])

In [20]:
matrix =([str(k) for k in indices.keys()], x0, sequential[0](x0))
col_widths = [max(len(custom_formatter(row1)), len(custom_formatter(row2)), len(name)) for name,row1,row2 in zip(*matrix)]
print(" ".join("{:<{}}".format(name, width) for name, width in zip(matrix[0], col_widths)))
for row in matrix[1:]:
    print(" ".join("{:<{}}".format(custom_formatter(num), width) for num, width in zip(row, col_widths)))

A_solar D_f alpha d   D_d  D_s h_f  t_d t_s V_d g   rho_w F_B F_W K_B t_f K_G I B_M G_M C_33 A_33 m_platform \omega_0 eta_solar m_solar m_batt m_comms m_prop m_struct rho rho_h S_w C_d eta_m v P_move c      f      l e_t theta_t L_pt eta_parab G_t D_r G_r R_e    h      S L_s E_N  L_a   L_l  L_p   R      T_s k        P_comms t_move E_move P_hotel t_mission E_hotel t_comms E_comms E_AUV G E_service t_service P_service E_required E_recharge t_recharge P_recharge I_deg L_solar d_deg eta_s phi_s theta DOD N_batt eta_battery C m_battzero mu_battery
1       2   0.2   0.3 1.25 2   0.45 0.5 2   1   9.8 1000  1   1   1   0.5 1   1 1   1   1    1    1          1        10        1       1      50      50     1        700 2700  1   1   0.75  1 1      3.00e8 2.20e9 1 27  32      1    0.55      1   5.3 1   6.38e6 6.00e5 1 1   38.9 0.933 0.79 0.933 8.50e7 135 1.38e-23 1       3600   1      50      8.64e4    1       3600    1       1900  1 1         4.32e4    1         1          1          4.32e4     

## Run with fixed parameters

In [304]:
objective = Function(((symb_str_mapping['m_platform'],), lambda m: m), indices=indices)
ineq_constraint = EliminateAnalysisMergeResiduals([], [
    Function(((symb_str_mapping['h_f'],symb_str_mapping['t_f']), lambda hf,tf: hf/(0.9*tf)-1), indices=indices),
    Function(((symb_str_mapping['D_s'],symb_str_mapping['D_f']), lambda Ds,Df: Ds/(0.9*Df)-1), indices=indices),
    Function(((symb_str_mapping['D_s'],symb_str_mapping['D_d']), lambda Ds,Dd: Ds/(0.9*Dd)-1), indices=indices),
    Function(((symb_str_mapping['P_move'],), lambda P: -P), indices=indices),
    Function(((symb_str_mapping['t_d'],), lambda td: -td/0.1+1), indices=indices),
])

In [305]:
ineq_constraint.structure

(tensor([ 6, 15,  5,  1,  4, 36,  7]), tensor([]))

In [306]:
idf_eval = ParallelAnalysis([all_sequential], [objective]+[ineq_constraint]+equality_constraints, sharedvars=couplingvars)

In [307]:
params = [2,3,10,11,23,24,26,27,28,29,30,31,33,34,37,38,39,40,41,42,43,45,47,48,51,52,53,54,55,56,57,59,61,62,64,66,67,69,73,75,76,77,78,79,80,81,82,83,85,86]

In [308]:
np.array([list(indices.keys())[elt] for elt in params])

array([alpha, d, g, rho_w, \omega_0, eta_solar, m_batt, m_comms, m_prop,
       m_struct, rho, rho_h, C_d, eta_m, c, f, l, e_t, theta_t, L_pt,
       eta_parab, D_r, R_e, h, E_N, L_a, L_l, L_p, R, T_s, k, t_move,
       P_hotel, t_mission, t_comms, E_AUV, G, t_service, t_recharge,
       I_deg, L_solar, d_deg, eta_s, phi_s, theta, DOD, N_batt,
       eta_battery, m_battzero, mu_battery], dtype=object)

In [309]:
for key,val in all_sequential.structure_full.items():
    print(key, [str(list(indices.keys())[elt]) for elt in val if elt not in params])

3 ['D_f']
9 ['t_s', 'h_f', 'D_d', 'D_s', 't_d', 'D_f']
12 ['t_s', 'h_f', 'D_d', 'D_s', 't_d', 'D_f']
13 ['t_s', 'h_f', 'D_d', 'D_s', 't_d', 'D_f']
14 ['t_s', 'D_d', 'D_f', 'D_s', 'h_f', 't_d']
16 ['t_s', 'D_d', 'D_f', 't_f', 'D_s', 't_d']
17 ['D_f']
18 ['D_f', 't_s', 'h_f', 'D_d', 'D_s', 't_d']
19 ['D_f', 't_s', 'h_f', 'D_d', 'D_s', 't_d', 't_f']
20 ['D_f']
21 ['D_s', 'D_d', 'D_f']
23 ['D_f', 'D_s', 'D_d', 'm_platform']
22 ['t_s', 'h_f', 'D_d', 'D_s', 't_d', 'D_f']
25 ['A_solar']
29 ['A_solar', 't_s', 'h_f', 'D_d', 'D_s', 't_d', 'D_f']
7 ['t_s', 'D_d', 'D_f', 't_f', 'D_s', 'A_solar', 'h_f', 't_d']
32 ['D_s', 'h_f', 'D_d', 't_s', 'D_f', 't_f', 'A_solar', 't_d']
36 ['D_s', 'h_f', 'D_d', 't_s', 'D_f', 't_f', 'A_solar', 't_d', 'v']
39 []
42 []
44 ['D_f']
46 []
49 []
50 []
58 ['D_f']
60 ['D_s', 'h_f', 'D_d', 't_s', 'D_f', 't_f', 'A_solar', 't_d', 'v']
63 []
65 ['D_f']
68 []
70 []
71 ['D_s', 'h_f', 'D_d', 't_s', 'D_f', 't_f', 'A_solar', 't_d', 'v']
72 ['D_s', 'h_f', 'D_d', 't_s', 'D_f', 't_f

In [310]:
xvalsdict = load_file('pearl_params')
x0 = torch.tensor([xvalsdict[str(key)] for key, val in indices.items()], dtype=torch.float64, requires_grad=True)
coupling_idx = torch.tensor(couplingvars)

In [311]:
idf_pure = ParallelAnalysis([all_sequential], [], sharedvars=couplingvars)

In [312]:
[(list(indices.keys())[elt],elt.item()) for elt in coupling_idx]

[(A_solar, 0), (t_d, 7), (m_batt, 26)]

In [313]:
eps = 1e-5
xn = x0 + torch.rand_like(x0)
xcopy = xn.detach()
((all_sequential(xcopy+eps*torch.eye(87)[:,0])-all_sequential(xcopy))/eps)[0]

tensor(0., dtype=torch.float64)

In [314]:
all_sequential(xcopy)[23]

tensor(2.3716, dtype=torch.float64)

In [315]:
couplingvars

[0, 7, 26]

In [316]:
flat_sets[0].analysis.structure

(tensor([2, 1]), tensor([3]))

In [317]:
import torch
from torch.autograd.functional import jacobian

test_idx = [0, 1, 4, 5, 6, 7, 8, 15, 22, 24, 35, 55] #coupling_idx
xcopy = x0.detach()
xinit = x0[test_idx]

def run_with_smaller(x):
    xcopy[test_idx] = x
    #out = sequential_all(xcopy[coupling_idx]
    #out = idf_pure(xcopy)[0]
    out = torch.cat(idf_eval(xcopy))
    return out

jacobian(run_with_smaller, xinit).numpy()

array([[0, 1413.72, 981.75, 6283.19, 3141.59, 1227.18, 3141.59, 0, 0, 0,
        0, 0],
       [0, 0, 0, 0, 2.22, 0, 0, -2, 0, 0, 0, 0],
       [0, -0.56, 0, 0.56, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, -1.42, 0.89, 0, 0, 0, 0, 0, 0, 0, 0],
       [7.9, -3285.1, -1685.95, -1489.35, -6671.04, -969.63, -4933.46,
        1737.57, 0, 0.79, -35817.21, 0],
       [0.03, -0.95, 10.08, -5.69, -9.48, -3.7, -2.84, 6.64, 0, 0, 0, 0],
       [0, 0.18, -0.03, -0.03, -0.12, -0.02, -0.09, 0.03, 0, 0, -0.66,
        -0],
       [-0.01, 0.19, -2.02, 1.14, 1.9, -2.52, 0.57, -1.33, 0, -0, 0, 0],
       [-10.35, 2.58, 1.33, 1.17, 5.25, 0.76, 3.88, -1.37, 0, -0, 28.18,
        0],
       [-0.07, 27.51, 14.17, 12.52, 56.06, 8.15, 41.46, -14.6, 0, -0.01,
        300.98, 0]])

In [318]:
len(idf_eval.structure_full)

10

In [319]:
def generate_optim_functions(optim_funcs, solvefor_indices, x):
    def eval_all(y):
        x.requires_grad_(False)
        x[solvefor_indices] = torch.from_numpy(y).to(x.dtype)
        objval, ineqval, eqval, res = optim_funcs(x)
        x.requires_grad_(True)
        return objval.item(), -ineqval, -torch.cat((eqval, res))
    
    def obj_function(y):
        objval, _, _ = eval_all(y)
        return objval

    def ineq_function(y=None):
        _, ineqval, _ = eval_all(y)
        return ineqval
    
    def eq_function(y=None):
        _, _, eqval = eval_all(y)
        return eqval
    
    def dobj(y):
        xc = x.detach() # remove gradient tracking
        xc[solvefor_indices] = torch.from_numpy(y).to(x.dtype)
        dobj = jacobian(lambda w: optim_funcs(w)[0], xc).numpy()[0]
        return dobj[solvefor_indices]

    def dineq(y=None):
        xc = x.detach()
        xc[solvefor_indices] = torch.from_numpy(y).to(x.dtype)
        dineq = jacobian(lambda w: optim_funcs(w)[1], xc).numpy()
        return -dineq[:,solvefor_indices]
    
    def deq(y=None):
        xc = x.detach()
        xc[solvefor_indices] = torch.from_numpy(y).to(x.dtype)
        deq = jacobian(lambda w: torch.cat(optim_funcs(w)[2:]), xc).numpy()
        return -deq[:,solvefor_indices]

    xguess = x[solvefor_indices].detach().numpy()
    
    return xguess, obj_function, ineq_function, eq_function, dobj, dineq, deq

In [320]:
for item in idf_eval.structure_full:
    print([elt for elt in item if elt not in params])

[0, 1, 4, 5, 6, 7, 8, 15, 22, 35]
[6, 15]
[5, 1]
[5, 4]
[0, 1, 4, 5, 6, 7, 8, 15, 22, 35]
[0, 1, 4, 5, 6, 7, 8, 15, 22, 35]
[0, 1, 4, 5, 6, 7, 8, 15, 22, 35]
[7, 0, 1, 4, 5, 6, 8, 15, 22, 35]
[0, 1, 4, 5, 6, 7, 8, 15, 22, 35]
[0, 1, 4, 5, 6, 7, 8, 15, 22, 35]


In [321]:
xvalsdict = load_file('pearl_params')
x0 = torch.tensor([xvalsdict[str(key)] for key, val in indices.items()], dtype=torch.float64)
numerical_values = {'D_f': 3.7, 'D_d':1., 'D_s':0.15, 't_d':0.2, 'h_f':0.09, 't_s':0.2}
for var, val in numerical_values.items():
    x0[indices[sp.symbols(var)]] = val
x0.requires_grad_(True)
designvars = [1,4,5,8,15]#1,8,4,5,15]
couplingvars = [0,7,26]
othervars = []
solvefor_indices = torch.tensor(designvars+othervars+couplingvars)
xguess, obj_function, ineq_function, eq_function, dobj, dineq, deq = generate_optim_functions(idf_eval, solvefor_indices, x0)
constraints = [{'type': 'eq', 'fun': eq_function, 'jac': deq}]
constraints.append({'type': 'ineq', 'fun': ineq_function, 'jac': dineq})
bnds_problem = [(0.1, 10) for _ in designvars]+[(0, None) for _ in othervars+couplingvars]

In [322]:
eps = 1e-6
xcopy = xguess
testf = ineq_function
(testf(xguess+eps*np.eye(len(xcopy))[:,1])-testf(xcopy))/eps #obj 4,6 zero gradient


tensor([0.0000e+00, 0.0000e+00, 1.6667e-01, 5.1191e+03, 2.7403e+01],
       dtype=torch.float64)

In [323]:
dineq(xguess)

array([[-0, -0, -0, -0, 0.4, -0, -0, -0],
       [0.01, -0, -0.3, -0, -0, -0, -0, -0],
       [-0, 0.17, -1.11, -0, -0, -0, -0, -0],
       [2570.68, 5119.15, 118.68, 319.4, -7433.55, -9.88, 775.7, -0.99],
       [-7.13, 27.4, 0.07, 0.03, -35.49, -0.05, 3.7, -0]])

In [324]:
inputs = torch.tensor([elt.item() for elt in all_sequential.structure[0] if elt not in all_sequential.structure[1]])
np.array([str(list(indices.keys())[inp]) for inp in inputs])

array(['D_f', 'alpha', 'D_d', 'D_s', 'h_f', 't_s', 'g', 'rho_w', 't_f',
       'eta_solar', 'm_comms', 'm_prop', 'rho', 'rho_h', 'C_d', 'eta_m',
       'v', 'c', 'f', 'e_t', 'theta_t', 'eta_parab', 'D_r', 'R_e', 'h',
       'E_N', 'L_a', 'L_l', 'L_p', 'R', 'T_s', 'k', 't_move', 'P_hotel',
       't_mission', 't_comms', 'E_AUV', 'G', 't_service', 't_recharge',
       'I_deg', 'L_solar', 'd_deg', 'eta_s', 'phi_s', 'theta', 'DOD',
       'N_batt', 'eta_battery', 'm_battzero', 'mu_battery'], dtype='<U11')

In [325]:
indices[symb_str_mapping['R']]

tensor([55])

In [326]:
[[str(k) for k in indices.keys()][elt] for elt in designvars], [[str(k) for k in indices.keys()][elt] for elt in couplingvars]

(['D_f', 'D_d', 'D_s', 't_s', 't_f'], ['A_solar', 't_d', 'm_batt'])

In [327]:
#xguess = np.array([1.98, 10, 0.21, 0.19, 0.1, 0.1, 585, 0.03, 20])

In [328]:
minimize(obj_function, xguess, bounds=bnds_problem, jac=dobj, 
         constraints=constraints, method='SLSQP', options={'maxiter':1000})

c:\Users\johan\miniconda3\envs\minimdo\lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1365.9152749593097
       x: [ 3.885e+00  4.044e-01  3.639e-01  2.752e+00  1.000e-01
            9.483e+00  1.000e-01  1.063e+02]
     nit: 40
     jac: [ 5.492e+02  6.352e+01  1.573e+03  1.040e+02  0.000e+00
            0.000e+00  1.284e+02  0.000e+00]
    nfev: 52
    njev: 40

In [329]:
x0.detach().numpy()

array([9.48, 3.88, 0.2, 0.3, 0.4, 0.36, 0.09, 0.1, 2.75, 1, 9.8, 1000, 1,
       1, 1, 0.1, 1, 1, 1, 1, 1, 1, 1, 1, 10, 1, 106.29, 50, 50, 1, 700,
       2700, 1, 1, 0.75, 1, 1, 300000000, 2200000000, 1, 27, 32, 1, 0.55,
       1, 5.3, 1, 6378000, 600000, 1, 1, 38.9, 0.93, 0.79, 0.93, 85000000,
       135, 0, 1, 3600, 1, 50, 86400, 1, 3600, 1, 1900, 1, 1, 43200, 1, 1,
       1, 43200, 1, 0.9, 1, 0.05, 0.27, 800, 0.96, 0.7, 1, 0.85, 1, 5,
       720000])

In [250]:
idf_eval(x0.detach())

[tensor([1346.1686], dtype=torch.float64),
 tensor([-9.9000e-01, -3.4117e-13, -9.7506e-01, -1.1635e+04,  5.4655e-11],
        dtype=torch.float64),
 tensor([8.1712e-14], dtype=torch.float64),
 tensor([-4.8590e-11, -8.1823e-14, -7.8271e-14], dtype=torch.float64)]

In [251]:
idf_eval(all_sequential(x0))

[tensor([1346.1686], dtype=torch.float64, grad_fn=<IndexBackward0>),
 tensor([-9.9000e-01, -3.4117e-13, -9.7506e-01, -1.1635e+04,  6.4256e-11],
        dtype=torch.float64, grad_fn=<CatBackward0>),
 tensor([9.7700e-14], dtype=torch.float64, grad_fn=<AddBackward0>),
 tensor([-9.6002e-12, -1.5876e-14, -1.4988e-14], dtype=torch.float64,
        grad_fn=<CatBackward0>)]

In [252]:
xsol = all_sequential(x0).detach().numpy()

In [253]:
def custom_formatter(x):
    if abs(x) < 0.01:
        return "{:.2e}".format(x).replace('+0', '')
    elif abs(x) > 10000:
        return "{:.2e}".format(x).replace('+0', '')
    else:
        return "{:.3f}".format(x).rstrip('0').rstrip('.')
    
matrix =([str(k) for k in indices.keys()], xsol)
col_widths = [max(len(custom_formatter(row1)), len(name)) for name,row1 in zip(*matrix)]
print(" ".join("{:<{}}".format(name, width) for name, width in zip(matrix[0], col_widths)))
for row in matrix[1:]:
    print(" ".join("{:<{}}".format(custom_formatter(num), width) for num, width in zip(row, col_widths)))

A_solar D_f  alpha d     D_d D_s   h_f  t_d t_s   V_d   g   rho_w F_B    F_W    K_B   t_f K_G I      B_M   G_M   C_33   A_33   m_platform \omega_0 eta_solar m_solar m_batt  m_comms m_prop m_struct rho rho_h S_w    C_d eta_m v P_move c      f      l     e_t theta_t L_pt eta_parab G_t    D_r G_r     R_e    h      S      L_s      E_N  L_a   L_l  L_p   R      T_s k        P_comms t_move E_move P_hotel t_mission E_hotel t_comms E_comms E_AUV G E_service t_service P_service E_required E_recharge t_recharge P_recharge I_deg L_solar d_deg eta_s phi_s theta DOD N_batt eta_battery C      m_battzero mu_battery
10.101  4.01 0.2   0.802 0.1 0.224 0.09 0.1 5.282 1.346 9.8 1000  1.32e4 1.32e4 3.346 0.1 3.4 12.686 9.424 9.371 1.24e5 1.06e4 1346.169   5.818    10        101.01  112.886 50      50     1032.272 700 2700  17.453 1   0.75  1 1.16e4 3.00e8 2.20e9 0.136 27  32      0.14 0.55      26.255 5.3 8200.06 6.38e6 6.00e5 2.83e6 1.47e-17 38.9 0.933 0.79 0.933 8.50e7 135 1.38e-23 2.833   3600   4.19e7 

In [29]:
noelim = ParallelAnalysis(list(sequential.values()), [])

In [45]:
torch.tensor(noelim.sharedvars)

tensor([ 2,  3,  9, 12, 13, 14, 16, 17, 18, 19, 20, 21, 23, 22, 25, 29,  8, 32,
        36, 39, 42, 44, 46, 49, 50, 58, 60, 63, 65, 68, 70, 71, 72, 74,  1, 84,
        26])

In [34]:
all_sequential = EliminateAnalysis(list(sequential.values()), [])

In [35]:
all_sequential.structure

(tensor([ 0,  1,  4,  5,  6,  7,  8, 10, 11, 15, 22, 24, 26, 27, 28, 30, 31, 33,
         34, 35, 37, 38, 40, 41, 43, 45, 47, 48, 51, 52, 53, 54, 55, 56, 57, 59,
         61, 62, 64, 66, 67, 69, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86]),
 tensor([ 2,  3,  9, 12, 13, 14, 16, 17, 18, 19, 20, 21, 23, 22, 25, 29,  8, 32,
         36, 39, 42, 44, 46, 49, 50, 58, 60, 63, 65, 68, 70, 71, 72, 74,  1, 84,
         26]))

In [36]:
couplingvars = [elt for elt in all_sequential.structure[0] if elt in all_sequential.structure[1]]
couplingvars

[tensor(1), tensor(8), tensor(22), tensor(26)]

In [39]:
residuals = ParallelAnalysis([all_sequential],[], sharedvars=couplingvars)

In [40]:
residuals.structure

(tensor([ 0,  1,  4,  5,  6,  7,  8, 10, 11, 15, 22, 24, 26, 27, 28, 30, 31, 33,
         34, 35, 37, 38, 40, 41, 43, 45, 47, 48, 51, 52, 53, 54, 55, 56, 57, 59,
         61, 62, 64, 66, 67, 69, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86]),
 ())

In [42]:
from graph.graphutils import all_variables

In [43]:
torch.manual_seed(43)
all_indices = torch.cat(list(indices.values()))
x0 = torch.zeros_like(all_indices, dtype=torch.float64)
# only set entries corresponding to variables specified in a dict e.g. {D_f:1.0}
# to the value specified in the dict
numerical_values = {'D_f': 1.0, 'D_d':1.0, 'D_s':1.0, 't_d':4/3, 'h_f':4/3, 't_s':4/3}
for var, val in numerical_values.items():
    x0[indices[sp.symbols(var)]] = val

In [47]:
from torchengine import EliminateAnalysis